# 🚢 Titanic Dataset Description 🚢

### TASK:

Recreate the Pandas-driven EDA with the Titanic dataset, but use PySpark in place of Pandas. Disregard visualizations.

### VARIABLE DESCRIPTIONS:

* survival ----- Survival (0 = No; 1 = Yes)
* pclass ------- Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name --------- Name
* sex ---------- Sex
* age ---------- Age
* sibsp -------- Number of Siblings/Spouses Aboard
* parch -------- Number of Parents/Children Aboard
* ticket ------- Ticket Number
* fare --------- Passenger Fare
* cabin -------- Cabin
* embarked ----- Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

### ✨ PySpark imports

In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD as lrSGD

### ✨ Instantiate SparkSession

In [2]:
spark = SparkSession.builder.appName("Python Spark regression!").config("spark.banana.config.option", "banana-value").getOrCreate()

### 👓  Look at the first 5️⃣ rows of the dataframe

In [26]:
df = spark.read.csv('titanic.csv', header=True, inferSchema=True)
df.take(5)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S'),
 Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S'),
 Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.05, Cabin=None, Embarked='S')]

In [27]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [28]:
df.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [29]:
df.explain()

== Physical Plan ==
*(1) FileScan csv [PassengerId#1129,Survived#1130,Pclass#1131,Name#1132,Sex#1133,Age#1134,SibSp#1135,Parch#1136,Ticket#1137,Fare#1138,Cabin#1139,Embarked#1140] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/cherishkim/Code/ds23-3/titanic_pyspark/titanic.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<PassengerId:int,Survived:int,Pclass:int,Name:string,Sex:string,Age:double,SibSp:int,Parch:...


In [30]:
len(df.columns)

12

### 📆 How many Age values are empty (or null)?

In [31]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



177.

### ⚤ Create a new column as gender; when Sex is female it is 0️⃣, when Sex is male it is 1️⃣

In [34]:
df = df.withColumn('Gender', when(df.Sex == 'female', 0).otherwise(1))
df.show(20)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|     1|
|          6|       0|  

In [35]:
len(df.columns)

13

### 📆 List all of the Ages that are not null.

In [37]:
df_age_exists = df.filter(df.Age.isNotNull())
df_age_exists.take(5)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S', Gender=1),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C', Gender=0),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S', Gender=0),
 Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S', Gender=0),
 Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.05, Cabin=None, Embarked='S', Gender=1)]

In [39]:
df_age_exists.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|     1|
|          7|       0|  

### 🅲 Slice the df for those whose Embarked section was 'C'.

In [40]:
df_c = df.filter(df.Embarked == 'C')
df_c.take(5)

[Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C', Gender=0),
 Row(PassengerId=10, Survived=1, Pclass=2, Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age=14.0, SibSp=1, Parch=0, Ticket='237736', Fare=30.0708, Cabin=None, Embarked='C', Gender=0),
 Row(PassengerId=20, Survived=1, Pclass=3, Name='Masselmani, Mrs. Fatima', Sex='female', Age=None, SibSp=0, Parch=0, Ticket='2649', Fare=7.225, Cabin=None, Embarked='C', Gender=0),
 Row(PassengerId=27, Survived=0, Pclass=3, Name='Emir, Mr. Farred Chehab', Sex='male', Age=None, SibSp=0, Parch=0, Ticket='2631', Fare=7.225, Cabin=None, Embarked='C', Gender=1),
 Row(PassengerId=31, Survived=0, Pclass=1, Name='Uruchurtu, Don. Manuel E', Sex='male', Age=40.0, SibSp=0, Parch=0, Ticket='PC 17601', Fare=27.7208, Cabin=None, Embarked='C', Gender=1)]

In [41]:
df_c.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-------------+--------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|       Ticket|    Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+-------------+--------+-----+--------+------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|     PC 17599| 71.2833|  C85|       C|     0|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|       237736| 30.0708| null|       C|     0|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|         2649|   7.225| null|       C|     0|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|         2631|   7.225| null|       C|     1|
|         31|       0|     1|Uruchurtu, Don. M...|  male|40.0|    0|    0|     PC 17601| 27.7208| null|       C|     1|
|         32|       1|     1|Spencer, Mr

### 🤔 Describe a specific column.

In [47]:
df.select('Ticket').distinct().show()

+----------------+
|          Ticket|
+----------------+
|          367230|
|       P/PP 3381|
|          244270|
|          363291|
|SOTON/OQ 3101317|
|           31418|
|           26360|
|            2700|
|           14313|
|          345763|
|       A/5 21172|
|          364500|
|           19877|
|          350029|
|          113800|
|      A/4. 39886|
|       C.A. 2673|
|          113807|
|        PC 17604|
|       C.A. 2315|
+----------------+
only showing top 20 rows



In [52]:
df.select('Ticket').printSchema()

root
 |-- Ticket: string (nullable = true)



In [53]:
df.select('Ticket').explain()

== Physical Plan ==
*(1) FileScan csv [Ticket#1137] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/cherishkim/Code/ds23-3/titanic_pyspark/titanic.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Ticket:string>


### 👯‍ How many unique values does 'Embarked' have?

In [56]:
df.select('Embarked').distinct().show()

+--------+
|Embarked|
+--------+
|       Q|
|    null|
|       C|
|       S|
+--------+



In [58]:
df.select('Embarked').printSchema()

root
 |-- Embarked: string (nullable = true)



In [59]:
df.select('Embarked').explain()

== Physical Plan ==
*(1) FileScan csv [Embarked#1140] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/cherishkim/Code/ds23-3/titanic_pyspark/titanic.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Embarked:string>


In [60]:
df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Gender|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------+
|        891|       2|     3| 891|  2| 88|    7|    7|   681| 248|  147|       3|     2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------+



It has 3 unique values.